In [16]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import IBMQ, QuantumCircuit
from qiskit.providers.ibmq import RunnerResult
from qiskit.tools.monitor import job_monitor
# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2022-03-18 03:58:03,648: Credentials are already in use. The existing account in the session will be replaced.


In [21]:
provider = IBMQ.get_provider(hub='ibm-q-pnnl', group='internal', project='default')
mybackend = provider.get_backend('ibmq_mumbai')
print(provider)
print(mybackend)
provider = IBMQ.get_provider(hub='ibm-q-pnnl', group='internal', project='default')
provider.backends()

<AccountProvider for IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>
ibmq_mumbai


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-pnnl', group='internal', project='default')>,
 <IBMQBackend('ibmq_guadalupe

In [22]:
rt = provider.runtime
provider.runtime.pprint_programs()

hello-world:
  Name: hello-world
  Description: A sample runtime program.
qaoa:
  Name: qaoa
  Description: Qiskit Runtime QAOA program
torch-train:
  Name: torch-train
  Description: A runtime program to train hybrid QNN models with Pytorch.
torch-infer:
  Name: torch-infer
  Description: A runtime program to run inference and calculate the scores of Hybrid QNN models with Pytorch.
qasm3-runner:
  Name: qasm3-runner
  Description: A runtime program that takes one or more circuits, converts them to OpenQASM3, compiles them, executes them, and optionally applies measurement error mitigation. This program can also take and execute one or more OpenQASM3 strings. Note that this program can only run on a backend that supports OpenQASM3.
sampler:
  Name: sampler
  Description: Generates quasi-probabilities by sampling quantum circuits.
estimator:
  Name: estimator
  Description: Calculates expectation values of quantum operators.
sample-expval:
  Name: sample-expval
  Description: A sample e

In [33]:
qc = QuantumCircuit.from_qasm_file('schedule3.qasm')
qc.measure_all()
# qc.draw(output='mpl')
from qiskit import QuantumCircuit, Aer, execute
simbackend = provider.get_backend('ibmq_qasm_simulator')
job = execute(qc, simbackend)
result = job.result()
print(result.get_counts())

{'000000000000000000000000000': 4000}


In [29]:
cplist = [i for i in range(27)]
print(cplist)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [34]:
from qiskit.tools.monitor import job_monitor
shots = 20000
params = provider.runtime.program(program_id="circuit-runner").parameters()
program_inputs = {
    'circuits': qc,
    'shots': shots,
    'optimization_level': 0,
    #'iterations': 5,
    # 'initial_layout': [0,1,2,3,4,5,6],
    'measurement_error_mitigation': True,
}
# params.circuits = circuit
# params.measurement_error_mitigation = True
# program_inputs = {
#     'iterations': 3
# }
# Configure backend options
options = {'backend_name': mybackend.name()}

# Execute the circuit using the "circuit-runner" program.
job = provider.runtime.run(program_id="circuit-runner",
                        options=options,
                        inputs=program_inputs)

# Get runtime job result.
# job_monitor(job)
result = job.result(decoder=RunnerResult)

counts = result.get_counts()
print(counts)


In [7]:
def crosstest( list ):
    q = QuantumRegister(7, 'q')
    c = ClassicalRegister(7, 'c')
    circuit2 = QuantumCircuit(q, c)
    circuit2.x(q[list[0]])
    circuit2.x(q[list[2]])
    circuit2.cx(q[list[0]], q[list[1]])
    #circuit2.barrier()
    circuit2.cx(q[list[2]], q[list[1]])
    circuit2.cx(q[list[2]], [list[3]])
    circuit2.measure(q[list[0]],c[list[0]])
    circuit2.measure(q[list[1]],c[list[1]])
    circuit2.measure(q[list[2]],c[list[2]])
    circuit2.measure(q[list[3]],c[list[3]])
    #circuit2.draw(output = 'mpl')
    string = "0000000"
    shots = 20000
    for i in list:
        string = string[:i] + '1' + string[i + 1:]
    string = string[:list[1]] + '0' + string[list[1] + 1:]
    string = string[::-1]
    print(string)
    
    params = provider.runtime.program(program_id="circuit-runner").parameters()
    program_inputs = {
        'circuits': circuit2,
        'shots': shots,
        'optimization_level': 0,
        #'iterations': 5,
        'initial_layout': [0,1,2,3,4,5,6],
        'measurement_error_mitigation': True,
    }
# params.circuits = circuit
# params.measurement_error_mitigation = True
# program_inputs = {
#     'iterations': 3
# }
# Configure backend options
    options = {'backend_name': mybackend.name()}

# Execute the circuit using the "circuit-runner" program.
    job = provider.runtime.run(program_id="circuit-runner",
                               options=options,
                               inputs=program_inputs)

# Get runtime job result.
    result = job.result(decoder=RunnerResult)
    
    counts = result.get_counts()
    print(counts)
    error_rate = 1 - counts[string] / shots
    return error_rate

In [8]:
def barriertest( list ):
    q = QuantumRegister(7, 'q')
    c = ClassicalRegister(7, 'c')
    circuit2 = QuantumCircuit(q, c)
    circuit2.x(q[list[0]])
    circuit2.barrier()
    circuit2.x(q[list[2]])
    circuit2.cx(q[list[0]], q[list[1]])
    circuit2.cx(q[list[2]], [list[3]])
    circuit2.measure(q[list[0]],c[list[0]])
    circuit2.measure(q[list[1]],c[list[1]])
    circuit2.measure(q[list[2]],c[list[2]])
    circuit2.measure(q[list[3]],c[list[3]])
    #circuit2.draw(output = 'mpl')
    string = "0000000"
    shots = 20000
    for i in list:
        string = string[:i] + '1' + string[i + 1:]
    #string = string[:list[1]] + '0' + string[list[1] + 1:]
    string = string[::-1]
    print(string)
    
    params = provider.runtime.program(program_id="circuit-runner").parameters()
    program_inputs = {
        'circuits': circuit2,
        'shots': shots,
        'optimization_level': 0,
        #'iterations': 5,
        'initial_layout': [0,1,2,3,4,5,6],
        'measurement_error_mitigation': True,
    }
# params.circuits = circuit
# params.measurement_error_mitigation = True
# program_inputs = {
#     'iterations': 3
# }
# Configure backend options
    options = {'backend_name': mybackend.name()}

# Execute the circuit using the "circuit-runner" program.
    job = provider.runtime.run(program_id="circuit-runner",
                               options=options,
                               inputs=program_inputs)

# Get runtime job result.
    result = job.result(decoder=RunnerResult)
    
    counts = result.get_counts()
    print(counts)
    error_rate = 1 - counts[string] / shots
    return error_rate

In [9]:
import qiskit
qiskit.__version__

'0.19.0'

In [ ]:
list = [1,3,5,6]
res = runtimetest(list)
print(res)

1101010


In [ ]:
crossres = crosstest(list)
print(crossres)

In [ ]:
from numpy import mean
list = [1,3,4,5]
nocross = []
cross = []
for x in range(10):
    cur = runtimetest(list)
    cur2 = barriertest(list)
    nocross.append(cur)
    cross.append(cur2)
    print(f"no cross error rate {cur}")
    print(f"cross error rate {cur2}")

print(f"no cross average error rate {mean(nocross)}")
print(f" cross average error rate {mean(cross)}")
